# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

**Assignment**
Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
   * Excel and Vensim are only supported on Windows
   * Vensim requires the DSS version of Vensim
   * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get [NetLogo 6.3.0](https://ccl.northwestern.edu/netlogo/download.shtml)
   * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
   * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).


|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes [example scripts](https://github.com/quaquel/EMAworkbench/tree/master/ema_workbench/examples) for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel).
  * This [tutorial](https://emaworkbench.readthedocs.io/en/latest/basic_tutorial.html) also shows a simple model in Python, Vensim and Excel connected to the workbench.

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function.

In [1]:
# Some imports you may need
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

#from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
#from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.samplers import LHSSampler
from ema_workbench.em_framework.salib_samplers import MorrisSampler, SobolSampler

from ema_workbench.analysis.plotting import lines, Density

/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:19: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:34: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)
/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/analysis/prim.py:30: ImportWarning: altair based interactive inspection not available
  warnings.warn(("altair based interactive " "inspection not available"), ImportWarning)


In [2]:
# Import the Python function
from model.pred_prey import PredPrey

from ema_workbench import Model, RealParameter, TimeSeriesOutcome # Constant #TimeSeriesOutcome #IntegerParameter, BinaryParameter, CategoricalParameter

model = Model('PreyPy', function=PredPrey)

# Specify uncertainties, names of parameters need to match key word in the function
model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035), #IntegerParameter, BinaryParameter, CategoricalParameter
                       RealParameter('predation_rate', 0.0005, 0.003),
                       RealParameter('predator_efficiency',0.001, 0.004),
                       RealParameter('predator_loss_rate',0.04, 0.08)]

# Set levers, one for each time step
#model.levers = [RealParameter('l'+str(i), 0, 0.1) for i in range(100)]

# Specify outcomes
model.outcomes = [TimeSeriesOutcome('TIME'), #time
                  TimeSeriesOutcome('predators'), #predator
                  TimeSeriesOutcome('prey')] #prey

#model.constants = [Constant('final_time', 365), 
#                    Constant('dt', 0.25)]

In [3]:
# Define uncertainties and outcomes


# Define model objects for the different implementations


In [3]:
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=50)

100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 165.39it/s]


In [4]:
from ema_workbench import MultiprocessingEvaluator

In [5]:
if __name__ == "__main__":
    ema_logging.log_to_stderr(level=ema_logging.INFO)

    model = ExcelModel("PreyX", wd="./model/", model_file="PredPrey.xlsx")
    # Specify uncertainties, names of parameters need to match key word in the function
    model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035), #IntegerParameter, BinaryParameter, CategoricalParameter
                        RealParameter('predation_rate', 0.0005, 0.003),
                        RealParameter('predator_efficiency',0.001, 0.004),
                        RealParameter('predator_loss_rate',0.04, 0.08)]

    # specification of the outcomes
    model.outcomes = [TimeSeriesOutcome('TIME'), #time
                    TimeSeriesOutcome('predators'), #predator
                    TimeSeriesOutcome('prey')] #prey

    # name of the sheet
    model.default_sheet = "Sheet1"

    with MultiprocessingEvaluator(model) as evaluator:
        results = perform_experiments(model, 100, reporting_interval=1, evaluator=evaluator)

[MainProcess/INFO] pool started with 4 workers
[MainProcess/INFO] performing 100 scenarios * 1 policies * 1 model(s) = 100 experiments
  0%|                                                  | 0/100 [00:00<?, ?it/s]/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:19: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:19: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:19: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/Users/Mira/opt/anaconda3/envs/py311/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:19: ImportWarning: vensim con

In [ ]:
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=50)

In [ ]:
import numpy as np

from ema_workbench import (
    RealParameter,
    ema_logging,
    ScalarOutcome,
    TimeSeriesOutcome,
    MultiprocessingEvaluator,
)
from ema_workbench.connectors.netlogo import NetLogoModel

if __name__ == "__main__":
    # turn on logging
    ema_logging.log_to_stderr(ema_logging.INFO)

    model = NetLogoModel(
        "predprey", wd="./models/predatorPreyNetlogo", model_file="Wolf Sheep Predation.nlogo"
    )
    model.run_length = 100
    model.replications = 10

    model.uncertainties = [
        RealParameter("grass-regrowth-time", 1, 99),
        RealParameter("initial-number-sheep", 50, 100),
        RealParameter("initial-number-wolves", 50, 100),
        RealParameter("sheep-reproduce", 5, 10),
        RealParameter("wolf-reproduce", 5, 10),
    ]

    model.outcomes = [
        ScalarOutcome("sheep", variable_name="count sheep", function=np.mean),
        TimeSeriesOutcome("wolves"),
        TimeSeriesOutcome("grass"),
    ]

    # perform experiments
    n = 10

    with MultiprocessingEvaluator(model, n_processes=2, maxtasksperchild=4) as evaluator:
        results = evaluator.perform_experiments(n)

    print()